In [1]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import pywt
import matlab.engine
import dippykit as dip
import ssim.ssimlib as pyssim
from PIL import Image
import glob
from keras.models import load_model
from skimage.exposure import match_histograms 
eng = matlab.engine.start_matlab()
eng.addpath("Function\InputWeights")
eng.addpath("Function\InputWeights_U")
eng.addpath("Function")


[2023-12-03 09:54:59,943] DEBUG: Falling back to TensorFlow client; we recommended you install the Cloud TPU client directly with pip install cloud-tpu-client.
[2023-12-03 09:55:00,510] DEBUG: Creating converter from 7 to 5
[2023-12-03 09:55:00,511] DEBUG: Creating converter from 5 to 7
[2023-12-03 09:55:00,511] DEBUG: Creating converter from 7 to 5
[2023-12-03 09:55:00,512] DEBUG: Creating converter from 5 to 7
[2023-12-03 09:55:04,385] WARNING: From c:\Users\ajayr\miniconda3\envs\dip_project\lib\site-packages\keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.



'C:\\Users\\ajayr\\Documents\\Masters\\3rd Sem\\Digital Image Processing\\Project\\Function\\InputWeights_U;C:\\Users\\ajayr\\Documents\\Masters\\3rd Sem\\Digital Image Processing\\Project\\Function\\InputWeights;C:\\Users\\ajayr\\Documents\\MATLAB;C:\\Program Files\\MATLAB\\R2023a\\toolbox\\matlab\\addon_enable_disable_management\\matlab;C:\\Program Files\\MATLAB\\R2023a\\toolbox\\matlab\\addon_updates\\matlab;C:\\Program Files\\MATLAB\\R2023a\\toolbox\\matlab\\addons;C:\\Program Files\\MATLAB\\R2023a\\toolbox\\matlab\\addons\\cef;C:\\Program Files\\MATLAB\\R2023a\\toolbox\\matlab\\addons\\fileexchange;C:\\Program Files\\MATLAB\\R2023a\\toolbox\\matlab\\addons\\supportpackages;C:\\Program Files\\MATLAB\\R2023a\\toolbox\\matlab\\addons_common\\matlab;C:\\Program Files\\MATLAB\\R2023a\\toolbox\\matlab\\addons_install_location\\matlab;C:\\Program Files\\MATLAB\\R2023a\\toolbox\\matlab\\addons_product;C:\\Program Files\\MATLAB\\R2023a\\toolbox\\matlab\\addons_product_support\\matlab;C:\\P

## Importing Dataset

In [2]:
Gt_path= r"Set12\Ground Truth\*"
Ca_path=r"Set12\ContrastAdjusted\*"
noise_path=r"Set12\Noise\*"
oe_path=r"Set12\Overexposure\*"
ue_path=r"Set12\Underexposure\*"

In [3]:
Gt_folder=glob.glob(Gt_path)
Ca_folder=glob.glob(Ca_path)
noise_folder = glob.glob(noise_path)
oe_folder=glob.glob(oe_path)
ue_folder=glob.glob(ue_path)

In [4]:

gt_imgs = []
ca_imgs = []
noise_imgs = []
oe_imgs = []
ue_imgs = []

for j in Gt_folder:
    gt_img = cv2.imread(j)
    gt_img = cv2.resize(gt_img, (256, 256))
    gt_imgs.append(gt_img)

for j in Ca_folder:
    ca_img = cv2.imread(j)
    ca_img = cv2.resize(ca_img, (256, 256))
    ca_imgs.append(ca_img)

for j in noise_folder:
    noise_img = cv2.imread(j)
    noise_img = cv2.resize(noise_img, (256, 256))
    noise_imgs.append(noise_img)

for j in oe_folder:
    oe_img = cv2.imread(j)
    oe_img = cv2.resize(oe_img, (256, 256))
    oe_imgs.append(oe_img)

for j in ue_folder:
    ue_img = cv2.imread(j)
    ue_img = cv2.resize(ue_img, (256, 256))
    ue_imgs.append(ue_img)

## Functions

In [5]:
def display_plots(gt_img,noisy_img,denoisy_img):
    # Create a figure with three subplots
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(12, 4))

    # Display the first image
    ax1.imshow(gt_img, cmap='gray')
    ax1.set_title('Ground Truth')
    ax1.axis('off')

    # Display the second image
    ax2.imshow(noisy_img, cmap='gray')
    ax2.set_title('Noisy Image')
    ax2.axis('off')

    # Display the third image
    ax3.imshow(denoisy_img, cmap='gray')
    ax3.set_title('Denoised Image')
    ax3.axis('off')

    # Adjust layout to prevent clipping of titles
    plt.tight_layout()

    # Show the plot
    plt.show()


In [6]:
def nlmeans_denoise(image, h=3, templateWindowSize=7, searchWindowSize=21):
    # Perform Non-Local Means Denoising using OpenCV
    return cv2.fastNlMeansDenoising(image, None, h, templateWindowSize, searchWindowSize)

def wavelet_denoise(img, threshold=20):

    channels = cv2.split(img)

    # Apply wavelet denoising to each channel independently
    denoised_channels = []
    for channel in channels:
        # Perform 2D wavelet transform
        coeffs = pywt.dwt2(channel, 'bior1.3')

        # Thresholding: Set small coefficients to zero
        coeffs = tuple(map(lambda x: pywt.threshold(x, threshold, mode='soft'), coeffs))

        # Inverse 2D wavelet transform
        denoised_channel = pywt.idwt2(coeffs, 'bior1.3')

        # Clip values to be in the valid range [0, 255]
        denoised_channel = np.clip(denoised_channel, 0, 255)

        # Convert to uint8 (required for image display)
        denoised_channel = denoised_channel.astype(np.uint8)

        denoised_channels.append(denoised_channel)

    # Merge the denoised channels back into an RGB image
    denoised_img = cv2.merge(denoised_channels)

    return denoised_img

def combine_denoising_methods(image):

    # Apply NLMeans denoising
    nlmeans_denoised = nlmeans_denoise(image)

    # Apply wavelet denoising
    wavelet_denoised = wavelet_denoise(image)

    nlmeans_denoised = np.array(nlmeans_denoised)
    wavelet_denoised = np.array(wavelet_denoised)

    nlmeans_denoised_norm = nlmeans_denoised / 255.0
    wavelet_denoised_norm = wavelet_denoised / 255.0


    # Combine the results (you can experiment with different blending strategies)
    combined_denoised_norm = 0.2 * nlmeans_denoised_norm + 0.8 * wavelet_denoised_norm

    combined_denoised = (combined_denoised_norm * 255).astype(np.uint8)

    return combined_denoised

In [7]:
def psnr_2_lsts(lst1,gt_lst2):
    psnrs = []
    for i,j in zip(lst1,gt_lst2):
        val = cv2.PSNR(j, i)
        psnrs.append(val)
    return psnrs

def plot_normal_distribution(old_psnrs, new_psnrs):
    # Calculate mean and standard deviation for each dataset
    mean1 = np.mean(old_psnrs)
    std_dev1 = np.std(old_psnrs)

    mean2 = np.mean(new_psnrs)
    std_dev2 = np.std(new_psnrs)

    # Create a range of x values
    x = np.linspace(min(old_psnrs + new_psnrs), max(old_psnrs + new_psnrs), 100)

    # Generate probability density functions (PDFs) for each dataset
    pdf1 = 1 / (std_dev1 * np.sqrt(2 * np.pi)) * np.exp(-(x - mean1)**2 / (2 * std_dev1**2))
    pdf2 = 1 / (std_dev2 * np.sqrt(2 * np.pi)) * np.exp(-(x - mean2)**2 / (2 * std_dev2**2))

    # Plot the normal distribution curves
    plt.plot(x, pdf1, label='old_psnrs', color='blue')
    plt.plot(x, pdf2, label='new_psnrs', color='red')

    plt.xlabel('X-axis')
    plt.ylabel('Probability Density')
    plt.title('Normal Distribution of Data')
    plt.legend()

    plt.show()

In [8]:
class Evaluation_Metrics:
    def __init__(self):
        pass  

    def psnr(self, img1, img2):
        return dip.metrics.PSNR(img1, img2)

    def mse(self, img1, img2):
        return dip.metrics.MSE(img1, img2)

    def ssim(self, img1, img2):
        img1=cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
        img2=cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)
        return dip.metrics.SSIM(img1, img2)[0]
    def cwssim(self,img1,img2):
        img1_pillow = Image.fromarray(img1)
        img2_pillow = Image.fromarray(img2)
        similarity = pyssim.SSIM(img1_pillow).cw_ssim_value(img2_pillow)
        return similarity
    def unique(self, img1, img2):
        return eng.mslUNIQUE(img1, img2)

    def ms_unique(self, img1, img2):
        return eng.mslMSUNIQUE(img1, img2)

    def summer(self, img1, img2):
        return eng.SUMMER(img1, img2)

    def csv(self, img1, img2):
        return eng.csv(img1, img2)
metrics=Evaluation_Metrics()

In [9]:
def evaluation_metrics(noise_type,denoisy_img,noisy_img,gt):
    eval_dict = {}
    eval_dict['Noise Type'] = noise_type
    # ******************* PSNR *******************
    psnr_noisy = metrics.psnr(noisy_img,gt)
    psnr_denoisy =metrics.psnr(denoisy_img,gt)
    eval_dict['PSNR_noisy'] = psnr_noisy
    eval_dict['PSNR_denoisy'] = psnr_denoisy

    # ******************* SSIM ******************* 
    ssim_noisy = metrics.ssim(noisy_img,gt)
    ssim_denoisy =metrics.ssim(denoisy_img,gt)
    eval_dict['SSIM_noisy'] = ssim_noisy
    eval_dict['SSIM_denoisy'] = ssim_denoisy

    # ******************* CW-SSIM ******************* 
    cw_ssim_noisy = metrics.cwssim(noisy_img,gt)
    cw_ssim_denoisy = metrics.cwssim(denoisy_img,gt)
    eval_dict['CW-SSIM_noisy'] = cw_ssim_noisy
    eval_dict['CW-SSIM_denoisy'] = ssim_denoisy
    # ******************* UNIQUE ******************* 
    unique_noisy = metrics.unique(noisy_img,gt)
    unique_denoisy =metrics.unique(denoisy_img,gt)
    eval_dict['unique_noisy'] = unique_noisy
    eval_dict['unique_denoisy'] = unique_denoisy

    # ******************* MS-UNIQUE ******************* 
    ms_unique_noisy = metrics.ms_unique(noisy_img,gt)
    ms_unique_denoisy =metrics.ms_unique(denoisy_img,gt)
    eval_dict['ms_unique_noisy'] = ms_unique_noisy
    eval_dict['ms_unique_denoisy'] = ms_unique_denoisy

    # ******************* CSV ******************* 
    csv_noisy = metrics.csv(noisy_img,gt)
    csv_denoisy =metrics.csv(denoisy_img,gt)
    eval_dict['csv_noisy'] = csv_noisy
    eval_dict['csv_denoisy'] = csv_denoisy
    # ******************* SUMMER *******************
    summer_noisy = metrics.summer(noisy_img,gt)
    summer_denoisy =metrics.summer(denoisy_img,gt)
    eval_dict['summer_noisy'] = summer_noisy
    eval_dict['summer_denoisy'] = summer_denoisy

    return eval_dict

def list_of_dicts_to_dataframe(data):
    df = pd.DataFrame(data)
    return df

## Wavelet + NLMeans

##### Contrast Adjusted

In [41]:
h = 10  # Filter strength. Higher values remove more noise but can also remove details.
tempWinSize = 7
searchWinSize = 21

# Loop through your list of images
comb_denoisy_imgs = []
for img in ca_imgs:
    # Apply NLMeans denoising to the current image
    denoised_img = combine_denoising_methods(img)
    comb_denoisy_imgs.append(denoised_img)
    del(img)  


In [42]:
empty=[]
for noi,denoi,gt in zip(ca_imgs[:50],comb_denoisy_imgs[:50],gt_imgs[:50]):
    empty.append(evaluation_metrics("Contrast Adjusted",denoi,noi,gt))

##### Noise

In [43]:
comb_denoisy_imgs = []
for img in noise_imgs:
    # Apply NLMeans denoising to the current image
    denoised_img = combine_denoising_methods(img)
    comb_denoisy_imgs.append(denoised_img)
    del(img)  

In [44]:
for noi,denoi,gt in zip(noise_imgs,comb_denoisy_imgs,gt_imgs):
    empty.append(evaluation_metrics("Noise",denoi,noi,gt))

##### Overexposure

In [45]:
comb_denoisy_imgs = []
for img in oe_imgs:
    # Apply NLMeans denoising to the current image
    denoised_img = combine_denoising_methods(img)
    comb_denoisy_imgs.append(denoised_img)
    del(img)  

In [46]:
for noi,denoi,gt in zip(oe_imgs,comb_denoisy_imgs,gt_imgs):
    empty.append(evaluation_metrics("Overexposure",denoi,noi,gt))

##### Underexposure

In [47]:
comb_denoisy_imgs = []
for img in ue_imgs:
    # Apply NLMeans denoising to the current image
    denoised_img = combine_denoising_methods(img)
    comb_denoisy_imgs.append(denoised_img)
    del(img)  

In [48]:
for noi,denoi,gt in zip(ue_imgs,comb_denoisy_imgs,gt_imgs):
    empty.append(evaluation_metrics("Underexposure",denoi,noi,gt))

In [49]:
Final_Data=list_of_dicts_to_dataframe(empty)
Final_Data

,Noise Type,PSNR_noisy,PSNR_denoisy,SSIM_noisy,SSIM_denoisy,CW-SSIM_noisy,CW-SSIM_denoisy,unique_noisy,unique_denoisy,ms_unique_noisy,ms_unique_denoisy,csv_noisy,csv_denoisy,summer_noisy,summer_denoisy
0,Contrast Adjusted,27.953763,28.046961,0.803499,0.783139,0.954076,0.783139,0.876466,0.846124,0.894136,0.876618,0.978196,0.975139,3.344733,3.466383
1,Contrast Adjusted,28.550408,27.664680,0.919771,0.872043,0.926085,0.872043,0.940067,0.938771,0.936901,0.936275,0.983035,0.974715,2.926440,2.886282
2,Contrast Adjusted,27.925831,28.029263,0.882467,0.840585,0.930306,0.840585,0.895083,0.875114,0.881841,0.863629,0.977753,0.971542,2.880180,2.906570
3,Contrast Adjusted,28.129006,28.077601,0.812708,0.770136,0.940465,0.770136,0.886571,0.874756,0.870100,0.861598,0.974604,0.969939,2.892172,2.924104
4,Contrast Adjusted,28.092818,27.879029,0.901794,0.857623,0.924290,0.857623,0.937552,0.936829,0.925237,0.922748,0.981555,0.972381,2.641101,2.654761
5,Contrast Adjusted,27.768569,29.645213,0.930617,0.901540,0.926251,0.901540,0.941556,0.941111,0.923256,0.921136,0.982820,0.973292,2.702902,2.629846
6,Contrast Adjusted,27.981517,28.529770,0.759779,0.732244,0.943603,0.732244,0.857324,0.827154,0.870831,0.850442,0.974234,0.970560,3.065864,3.124183
7,Contrast Adjusted,28.195501,28.491929,0.892066,0.839635,0.924231,0.839635,0.938852,0.938713,0.923416,0.923760,0.982807,0.974313,2.734379,2.730193
8,Contrast Adjusted,28.228401,28.136139,0.848654,0.807590,0.931527,0.807590,0.930257,0.918156,0.912039,0.900785,0.978255,0.971531,2.761575,2.806080
9,Contrast Adjusted,28.627660,29.634734,0.871620,0.840321,0.925123,0.840321,0.909367,0.892667,0.893359,0.879891,0.979308,0.974038,2.942597,2.947966


In [50]:
Final_Data.to_csv(r"C:\Users\ajayr\Documents\Masters\3rd Sem\Digital Image Processing\Project\Results\Denoising\Wavelet+NLMeans\Set12\Set12_results.csv")

## Deep Learning

In [20]:
model_path = 'Models\model_TSR.h5'
model = load_model(model_path)

[2023-12-03 09:56:36,082] DEBUG: Creating converter from 3 to 5


[2023-12-03 09:56:36,184] WARNING: From c:\Users\ajayr\miniconda3\envs\dip_project\lib\site-packages\keras\src\backend.py:1398: The name tf.executing_eagerly_outside_functions is deprecated. Please use tf.compat.v1.executing_eagerly_outside_functions instead.

[2023-12-03 09:56:36,227] WARNING: From c:\Users\ajayr\miniconda3\envs\dip_project\lib\site-packages\keras\src\layers\pooling\max_pooling2d.py:161: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



##### Contrast Adjusted

In [21]:
comb_denoisy_imgs = []
for img in ca_imgs:
    img=(img-np.min(img))/(np.max(img)-np.min(img))
    img=np.reshape(img, (1, 256, 256, 3))
    denoised_img = model.predict(img)
    denoised_img=np.reshape(denoised_img, (256, 256, 3))
    denoised_img = (denoised_img * 255).astype(np.uint8)
    comb_denoisy_imgs.append(denoised_img)

1/1 [==============================] - 0s 123ms/step


In [22]:
empty=[]
for noi,denoi,gt in zip(ca_imgs,comb_denoisy_imgs,gt_imgs):
    empty.append(evaluation_metrics("Contrast Adjusted",np.array(denoi),np.array(noi),np.array(gt)))

##### Noise

In [23]:
comb_denoisy_imgs = []
for img in noise_imgs:
    img=(img-np.min(img))/(np.max(img)-np.min(img))
    img=np.reshape(img, (1, 256, 256, 3))
    denoised_img = model.predict(img)
    denoised_img=np.reshape(denoised_img, (256, 256, 3))
    denoised_img = (denoised_img * 255).astype(np.uint8)
    comb_denoisy_imgs.append(denoised_img)

1/1 [==============================] - 0s 130ms/step


In [24]:
for noi,denoi,gt in zip(noise_imgs,comb_denoisy_imgs,gt_imgs):
    empty.append(evaluation_metrics("Noise",np.array(denoi),np.array(noi),np.array(gt)))

##### Overexposure

In [25]:
comb_denoisy_imgs = []
for img in oe_imgs:
    img=(img-np.min(img))/(np.max(img)-np.min(img))
    img=np.reshape(img, (1, 256, 256, 3))
    denoised_img = model.predict(img)
    denoised_img=np.reshape(denoised_img, (256, 256, 3))
    denoised_img = (denoised_img * 255).astype(np.uint8)
    comb_denoisy_imgs.append(denoised_img)

1/1 [==============================] - 0s 115ms/step


In [26]:
for noi,denoi,gt in zip(oe_imgs,comb_denoisy_imgs,gt_imgs):
    empty.append(evaluation_metrics("Overexposure",np.array(denoi),np.array(noi),np.array(gt)))

##### Underexposure

In [27]:
comb_denoisy_imgs = []
for img in ue_imgs:
    img=(img-np.min(img))/(np.max(img)-np.min(img))
    img=np.reshape(img, (1, 256, 256, 3))
    denoised_img = model.predict(img)
    denoised_img=np.reshape(denoised_img, (256, 256, 3))
    denoised_img = (denoised_img * 255).astype(np.uint8)
    comb_denoisy_imgs.append(denoised_img)

1/1 [==============================] - 0s 120ms/step


In [28]:
for noi,denoi,gt in zip(ue_imgs,comb_denoisy_imgs,gt_imgs):
    empty.append(evaluation_metrics("Underexposure",np.array(denoi),np.array(noi),np.array(gt)))

In [29]:
Final_Data=list_of_dicts_to_dataframe(empty)
Final_Data

,Noise Type,PSNR_noisy,PSNR_denoisy,SSIM_noisy,SSIM_denoisy,CW-SSIM_noisy,CW-SSIM_denoisy,unique_noisy,unique_denoisy,ms_unique_noisy,ms_unique_denoisy,csv_noisy,csv_denoisy,summer_noisy,summer_denoisy
0,Contrast Adjusted,27.953763,27.611386,0.803499,0.362620,0.954076,0.362620,0.876466,2.339993e-04,0.894136,0.045656,0.978196,0.939921,3.344733,4.982888
1,Contrast Adjusted,28.550408,28.308623,0.919771,0.492595,0.926085,0.492595,0.940067,4.557045e-07,0.936901,0.017868,0.983035,0.941960,2.926440,4.980461
2,Contrast Adjusted,27.925831,27.836871,0.882467,0.363165,0.930306,0.363165,0.895083,2.609887e-07,0.881841,0.010184,0.977753,0.932235,2.880180,4.990903
3,Contrast Adjusted,28.129006,27.960526,0.812708,0.257439,0.940465,0.257439,0.886571,1.224845e-03,0.870100,0.049542,0.974604,0.934885,2.892172,4.995001
4,Contrast Adjusted,28.092818,27.885011,0.901794,0.300051,0.924290,0.300051,0.937552,1.663029e-11,0.925237,0.003801,0.981555,0.923216,2.641101,4.987798
5,Contrast Adjusted,27.768569,27.998379,0.930617,0.348024,0.926251,0.348024,0.941556,4.115721e-08,0.923256,0.008535,0.982820,0.929305,2.702902,4.803662
6,Contrast Adjusted,27.981517,27.702077,0.759779,0.328163,0.943603,0.328163,0.857324,1.081129e-05,0.870831,0.021474,0.974234,0.936648,3.065864,4.972901
7,Contrast Adjusted,28.195501,28.001433,0.892066,0.372258,0.924231,0.372258,0.938852,4.039465e-05,0.923416,0.028907,0.982807,0.939595,2.734379,4.969068
8,Contrast Adjusted,28.228401,27.873087,0.848654,0.278291,0.931527,0.278291,0.930257,3.577378e-04,0.912039,0.046234,0.978255,0.935208,2.761575,4.984438
9,Contrast Adjusted,28.627660,27.817158,0.871620,0.306161,0.925123,0.306161,0.909367,2.012913e-04,0.893359,0.038745,0.979308,0.944281,2.942597,4.938161


In [30]:
Final_Data.to_csv(r"Results\Denoising\Deep learning\Set12\Set12_results.csv")

## Histogram Matching 

##### Contrast Adjusted

In [31]:
comb_denoisy_imgs = []
for i in range(len(ca_imgs)):
    denoised_img = match_histograms(ca_imgs[i],gt_imgs[i], channel_axis=-1)
    comb_denoisy_imgs.append(denoised_img)

In [32]:
empty=[]
for noi,denoi,gt in zip(ca_imgs,comb_denoisy_imgs,gt_imgs):
    empty.append(evaluation_metrics("Contrast Adjusted",np.array(denoi),np.array(noi),np.array(gt)))

##### Noise

In [33]:
comb_denoisy_imgs = []
for i in range(len(noise_imgs)):
    denoised_img = match_histograms(noise_imgs[i],gt_imgs[i], channel_axis=-1)
    comb_denoisy_imgs.append(denoised_img)

In [34]:
for noi,denoi,gt in zip(noise_imgs,comb_denoisy_imgs,gt_imgs):
    empty.append(evaluation_metrics("Noise",np.array(denoi),np.array(noi),np.array(gt)))

##### Overexposure

In [35]:
comb_denoisy_imgs = []
for i in range(len(oe_imgs)):
    denoised_img = match_histograms(oe_imgs[i],gt_imgs[i], channel_axis=-1)
    comb_denoisy_imgs.append(denoised_img)

In [36]:
for noi,denoi,gt in zip(oe_imgs,comb_denoisy_imgs,gt_imgs):
    empty.append(evaluation_metrics("Overexposure",np.array(denoi),np.array(noi),np.array(gt)))

##### Underexposure

In [37]:
comb_denoisy_imgs = []
for i in range(len(ue_imgs)):
    denoised_img = match_histograms(ue_imgs[i],gt_imgs[i], channel_axis=-1)
    comb_denoisy_imgs.append(denoised_img)

In [38]:
for noi,denoi,gt in zip(ue_imgs,comb_denoisy_imgs,gt_imgs):
    empty.append(evaluation_metrics("Underexposure",np.array(denoi),np.array(noi),np.array(gt)))

In [39]:
Final_Data=list_of_dicts_to_dataframe(empty)
Final_Data

,Noise Type,PSNR_noisy,PSNR_denoisy,SSIM_noisy,SSIM_denoisy,CW-SSIM_noisy,CW-SSIM_denoisy,unique_noisy,unique_denoisy,ms_unique_noisy,ms_unique_denoisy,csv_noisy,csv_denoisy,summer_noisy,summer_denoisy
0,Contrast Adjusted,27.953763,33.943615,0.803499,0.917324,0.954076,0.917324,0.876466,0.899465,0.894136,0.920883,0.978196,0.978149,3.344733,4.486882
1,Contrast Adjusted,28.550408,43.071963,0.919771,0.997503,0.926085,0.997503,0.940067,0.989002,0.936901,0.986104,0.983035,0.993495,2.926440,4.563612
2,Contrast Adjusted,27.925831,37.706707,0.882467,0.967017,0.930306,0.967017,0.895083,0.900955,0.881841,0.896121,0.977753,0.983497,2.880180,4.240131
3,Contrast Adjusted,28.129006,34.808275,0.812708,0.965132,0.940465,0.965132,0.886571,0.918557,0.870100,0.910606,0.974604,0.982152,2.892172,3.996729
4,Contrast Adjusted,28.092818,41.742677,0.901794,0.993376,0.924290,0.993376,0.937552,0.977525,0.925237,0.968741,0.981555,0.991213,2.641101,4.402544
5,Contrast Adjusted,27.768569,45.074165,0.930617,0.997498,0.926251,0.997498,0.941556,0.989142,0.923256,0.983350,0.982820,0.991329,2.702902,4.527342
6,Contrast Adjusted,27.981517,32.751437,0.759779,0.908458,0.943603,0.908458,0.857324,0.834999,0.870831,0.863102,0.974234,0.977500,3.065864,4.061785
7,Contrast Adjusted,28.195501,43.525680,0.892066,0.988559,0.924231,0.988559,0.938852,0.974223,0.923416,0.969968,0.982807,0.990590,2.734379,4.497724
8,Contrast Adjusted,28.228401,39.942726,0.848654,0.978679,0.931527,0.978679,0.930257,0.958462,0.912039,0.951359,0.978255,0.986605,2.761575,4.451829
9,Contrast Adjusted,28.627660,38.773566,0.871620,0.982271,0.925123,0.982271,0.909367,0.957427,0.893359,0.955839,0.979308,0.985725,2.942597,4.453504


In [40]:
Final_Data.to_csv(r"C:\Users\ajayr\Documents\Masters\3rd Sem\Digital Image Processing\Project\Results\Enhancement\Set12\Set12_results.csv")